# Problem

CafeVan is an application in which users can use their
phones to order Ice Creams! Yummy! The startup company
has lots of IceCream vans all over the city..

However, There has been recent complaints about driving
habits of CafeVan drivers! Not only that.. CafeVan drivers
have been asking for extra wage when they are in traffic..

In order to migitate current operation problems, our CTO
has a brillient idea of building monitoring apps on top
of our current MicroService Architecture!

Luckily for us.. The vans are all equipped with GPS tracker
devices that send velocity of vehicle over time! We are tasked
with building micro-services that penalties speeding drivers
and gives bonus to drivers stuck in traffic!

So we pick our laptops and head over to meeting room!

* CTO: So we are going to build two new micro-services!
* Confused Programmer: But we a single queue in which tracker information comes in?
* CTO: heh! Didn't you know? RabbitMQ can do Publisher/Subscriber via fanouts!
* Confused Programmer: Really? How Cool!
* CTP: Yaah! I have a brillient idea.. Let's call our new microservices `overspeed` and `jammed`!

So our beloved CTO heads over to describe what exchanges are!

So.. We are going to headstart to exchanges and queues and
describe various methods of communication RabbitMQ provides!

Exchanges are like Post Offices... They distribute messages,
and queues are PostBox'es... Typically you have 3 scenarios:

1. You are a nosy little guy/gal who wants to know everything going on!
2. You are an efficient guy/gal who wants to know a subset of information important to you.. like news about discounts in your district
3. You are a lonely person who check your PostBox everyday for letters from your distant relatives!

These typical situtations are manifested in:

1. Fanout Exchanges: They distribute messages to every queue bound to them!
2. Topic Exchanges: They distribute messages to queues whom match routing key!
3. Direct Exchanges: They distribute messages only to designated queue

So.. In our case.. We build our microservice architecture by first directing
all our GPS data into a fanout exchange, and then define each microservice
as a seperate **bound** queue to that fanout exchange!

Finally.. we have a microservice called **punisher** which we send van information
and amount (positive is punishment, negative is reward).

## Question: Do you use Fanout or Direct exchanges for **punisher** microservice and why?

:-?

## Input

The input stream is as follows: each message is a string encoded as `VanName:time:speed`. Time is measured as seconds passed after the initial release of application (we have our own epoch!‌:D)

Sample Input is as follows:

```
A:10:40
A:11:45
...
```

## Output

If a van is driving more than 60 km/h punish him/her with -1. If driving lower than 10km/h reward him/her with 1.

So if a van is speeding, output `A:-1` which A is van name, and if is stuck in traffic, simply output `A:1` to punisher microservice!

## Hint

Exchanges can be defined and declared as follows:

```
exch = kombu.Exchange(name='van-speed', type='fanout', durable=True)
queue = kombu.Queue(name='hello', exchange=exch, durable=True)

exch(ch).declare()
queue(ch).declare()
```

In [1]:
input = ['A:0:10', 'A:1:20', 'A:2:5', 'A:3:5', 'A:4:35', 'A:4:60', 'B:0:40', 'B:1:50', 'B:2:60', 'B:3:70', 'B:4:75']

# Setup Connection, Declare exchanges and Queues!

In [2]:
import kombu
from kombu.utils.url import as_url

url = as_url(scheme='amqp', host='rabbitmq', port=5672, user='guest', password='guest')
conn = kombu.Connection(url)
conn = kombu.Connection(url)
conn.connect()
ch = conn.channel()
exch = kombu.Exchange(name='van-speed', type='fanout', durable=True)
over_speed_queue = kombu.Queue(name='over_speed', exchange=exch, durable=True)
jammed = kombu.Queue(name='jammed', exchange=exch, durable=True)
exch(ch).declare()
over_speed_queue(ch).declare()
jammed(ch).declare()

result_queue = kombu.Queue(name='result', durable=True)
result_queue(ch).declare()



'result'

# Publish input data to input queue!

In [3]:
producer = kombu.Producer(ch)
for msg in input:
    producer.publish(msg, exchange=exch)

# Create **overspeed** microservice which responds to high velocities!

In [4]:
def over_speed_call_back(body,msg):
    splited_body = body.split(':')
    speed = int(splited_body[2])
    van_name = splited_body[0]
    if speed>60:
        result='{}:-1'.format(van_name)
        producer.publish(result, routing_key='result')
    msg.ack()
consumer = kombu.Consumer(channel=ch, queues=over_speed_queue, callbacks=[over_speed_call_back])
consumer.consume()


# Create **jammed** microservice which responds to low velocities!

In [5]:
def jammed_call_back(body,msg):
    splited_body = body.split(':')
    speed = int(splited_body[2])
    van_name = splited_body[0]
    if speed<10:
        result='{}:1'.format(van_name)
        producer.publish(result, routing_key='result')
    msg.ack()
consumer = kombu.Consumer(channel=ch, queues=jammed, callbacks=[jammed_call_back])
consumer.consume()

In [6]:
while True:
    conn.drain_events(timeout=1)

timeout: timed out